<a href="https://colab.research.google.com/github/francoishcm/BackTesting/blob/master/Zipline_Reloaded_BacktestEngine_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧰# **CODE TO INSTALL MODULES & PACKAGES**

In [1]:
# Mount Google Drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install ta-lib v0.4.0
%%bash
wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
tar -xzf ta-lib-0.4.0-src.tar.gz
cd ta-lib/
./configure
make
make install

In [ ]:
# Install zipline
%pip install zipline-reloaded

In [ ]:
# Install matplot lirary
!pip install matplotlib

In [ ]:
# Install data bundle 'Quandl'
!pip install quandl

In [ ]:
# Ingest Quandl using API key
!QUANDL_API_KEY=KUnssHvVERHb5XYu9C1- zipline ingest -b 'quandl'

In [ ]:
# Confirm existing bundle that were ingested
!zipline bundles

In [ ]:
# Install stockstats
!pip install stockstats

In [ ]:
# Install Pyfolio
!pip install git+https://github.com/quantopian/pyfolio

# 💽# **CODE TO SET AND CONFIRM CURRENT WORKING DIRECTORIES**

In [ ]:
import os 

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/content/drive/My Drive/"


In [ ]:
import os 

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!


# choose where you want your project files to be saved
project_folder = "Colab Notebooks/My Project Folder"


In [ ]:
def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  !touch 'new_file_in_working_directory.txt'
  print('\nYour working directory was changed to ' + root_dir + project_folder + \
        "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

create_and_set_working_directory(project_folder)

In [8]:
# Confirm current working directory
!pwd

/content


In [ ]:
import os
for f in os.listdir("/root/content/drive/MyDrive/Colab Notebooks/My Project Folder"):
	print(f)

# ⏳# **RUN CODE TO INGEST CUSTOM BUNDLES HERE** **bold text**

In [ ]:
# Ingest custom bundle
!zipline ingest --bundle 'custom_quandl'

In [ ]:
# Confirm existing bundles
!zipline bundles

# **CLENOW CORE TREND MODEL CODE** *Do Not Delete*

In [ ]:
#@title
%matplotlib inline

import zipline
from zipline import run_algorithm
from zipline.api import order_target_percent, symbol, set_commission, set_slippage, schedule_function, date_rules, time_rules
from datetime import date, datetime
import pytz
import matplotlib.pyplot as plt 
import pyfolio as pf
import pandas as pd 
import numpy as np 
from scipy import stats
from zipline.finance.commission import PerDollar
from zipline.finance.slippage import VolumeShareSlippage, FixedSlippage

#Model Settings

intial_portfolio = 100000
momentum_window = 125
minimum_momentum = 40
portfolio_size = 30
vola_window = 20

#Commission and Slippage Settings

enable_commission = True 
commission_pct = 0.001 
enable_slippage = True 
slippage_volume_limit = 0.025
slippage_impact = 0.05

def momentum_score(ts):

  #Input: Price time series.Output: Annualized exponential regression slope, multiplied by the R2

  # Make a list of consecutive numbers 
  x = np.arange(len(ts))
  # Get logs
  log_ts = np.log(ts)
  # Calculate regression values
  slope, intercept, r_value, p_value, std_err = stats.linregress(x, log_ts) 
  # Annualize percent
  annualized_slope = (np.power(np.exp(slope), 252) - 1) * 100 
  #Adjust for fitness
  score = annualized_slope * (r_value ** 2) 
  return score

def volatility(ts):
  return ts.pct_change().rolling(vola_window).std().iloc[-1]

def output_progress(context):

  #Output some performance numbers during backtest run 
  #This code just prints out the past month's performance,
  # so that we have something to look at while the backtest runs.

  # Get today's date
  today = zipline.api.get_datetime().date()

  # Calculate percent difference since last month
  perf_pct = (context.portfolio.portfolio_value / context.last_month) - 1

  # Print performance, format as percent with two decimals. 
  print("{} - Last Month Result: {:.2%}".format(today, perf_pct))

  # Remember today's portfolio value for next month's calculation 
  context.last_month = context.portfolio.portfolio_value

#Initialization and trading logic


def initialize(context):

  # Set commission and slippage.
  if enable_commission:
    comm_model = PerDollar(cost=commission_pct) 
  else:
    comm_model = PerDollar(cost=0.0) 
  set_commission(comm_model)
  if enable_slippage: slippage_model=VolumeShareSlippage(volume_limit=slippage_volume_limit,
price_impact=slippage_impact) 
  else:
    slippage_model=FixedSlippage(spread=0.0) 
  set_slippage(slippage_model)

  # Used only for progress output. 
  context.last_month = intial_portfolio

  # Store index membership
  context.index_members = pd.read_csv('/content/drive/MyDrive/index_members/sp500.csv', index_col=0, parse_dates=[0])


  #Schedule rebalance monthly. 
  schedule_function(
    func=rebalance, 
    date_rule=date_rules.month_start(), 
    time_rule=time_rules.market_open()
  )

def rebalance(context, data):
  # Write some progress output during the backtest 
  output_progress(context)

  # First, get today's date
  today = zipline.api.get_datetime()

  # Second, get the index makeup for all days prior to today.
  all_prior = context.index_members.loc[context.index_members.index < today]

  # Now let's snag the first column of the last, i.e. latest, entry. 
  latest_day = all_prior.iloc[-1,0]

  # Split the text string with tickers into a list 
  list_of_tickers = latest_day.split(',')

  # Finally, get the Zipline symbols for the tickers 
  todays_universe = [symbol(ticker) for ticker in list_of_tickers]

  todays_universe = [ 
    symbol(ticker) for ticker in
    context.index_members.loc[context.index_members.index < today].iloc[-1,0].split(',')
  ]

  # Get historical data
  hist = data.history(todays_universe, "close", momentum_window, "1d")

  # Make momentum ranking table
  ranking_table = hist.apply(momentum_score).sort_values(ascending=False)

  #Sell Logic
  #First we check if any existing position should be sold.
    #Sell if stock is no longer part of index.
    #Sell if stock has too low momentum value.

  kept_positions = list(context.portfolio.positions.keys()) 
  for security in context.portfolio.positions:
    if (security not in todays_universe): 
      order_target_percent(security, 0.0) 
      kept_positions.remove(security)
    elif ranking_table[security] < minimum_momentum: 
      order_target_percent(security, 0.0) 
      kept_positions.remove(security)

  #Stock Selection Logic
  #Check how many stocks we are keeping from last month.
  #Fill from top of ranking list, until we reach the desired total number of portfolio holdings.

  replacement_stocks = portfolio_size - len(kept_positions) 
  buy_list = ranking_table.loc[
    ~ranking_table.index.isin(kept_positions)][:replacement_stocks]
  new_portfolio = pd.concat(
    (buy_list,
    ranking_table.loc[ranking_table.index.isin(kept_positions)])
  )
  buy_list = ranking_table.loc[
    ~ranking_table.index.isin(kept_positions)][:replacement_stocks]

  #Calculate inverse volatility for stocks, and make target position weights.

  vola_table = hist[new_portfolio.index].apply(volatility) 
  inv_vola_table = 1 / vola_table
  sum_inv_vola = np.sum(inv_vola_table) 
  vola_target_weights = inv_vola_table / sum_inv_vola
  for security, rank in new_portfolio.iteritems(): 
    weight = vola_target_weights[security]
    if security in kept_positions:
      order_target_percent(security, weight)
    else:
      if ranking_table[security] > minimum_momentum: 
        order_target_percent(security, weight)

def analyze(context, perf):
  perf['max'] = perf.portfolio_value.cummax() 
  perf['dd'] = (perf.portfolio_value / perf['max']) - 1 
  maxdd = perf['dd'].min()

  ann_ret = (np.power((perf.portfolio_value.iloc[-1] / perf.portfolio_value.iloc[0]),(252 / len(perf)))) - 1

  print("Annualized Return: {:.2%} Max Drawdown: {:.2%}".format(ann_ret, maxdd))

  return


start = pd.to_datetime('2000-1-1', utc=True)
end = pd.to_datetime('2018-1-1', utc=True)

results = run_algorithm(
    start=start,
    end=end, 
    initialize=initialize, 
    analyze=analyze, 
    capital_base=intial_portfolio, 
    data_frequency = 'daily', 
    bundle='quandl')

# 💻# **RUN BACKTEST CODE HERE**

In [2]:
#@title
%matplotlib inline

from zipline import run_algorithm
from zipline.api import order_target_percent, symbol, record, set_benchmark, set_commission
from zipline.finance.commission import PerTrade
from datetime import datetime
import pytz
import pandas as pd
import pyfolio as pf
import numpy as np
from stockstats import StockDataFrame as sdf
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display
import ipywidgets as widgets
out = widgets.HTML()
display(out)

def report_result(context, data):
  context.months += 1
  today = zipline.api.get_datetime().date()
  # Calculate annualized return so far
  ann_ret = np.power(context.portfolio.portfolio_value / starting_portfolio, 12 / context.months) - 1
  # Update the text
  out.value = """{} We have traded <b>{}</b> months and the annualized return is <b>{:.2%}</b>""".format(today, context.months, ann_ret)

def initialize(context):
  context.stock = symbol('IBM')
  context.rolling_window = 200
  set_benchmark(symbol('IBM'))

def handle_data(context, data):
  
  price_hist = data.history(context.stock, "close", context.rolling_window, "1d")
  
  order_target_percent(context.stock, 1.0 if price_hist[-1] > price_hist.mean() else 0.0)

def analyze(context, perf):

  # We need to be able to calulate the daily returns from the cumulative returns

  daily_returns = pd.Series([0.5, -0.5, 0.5, -0.5])
  cumulative_returns = pd.Series([0.5, -0.25, 0.125, 0.5625])

  returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(perf)
  benchmark_period_return = perf['benchmark_period_return']

  # Convert benchmark returns to daily returns
  #daily_returns = (1 + benchmark_period_return) / (1 + benchmark_period_return.shift()) - 1
  daily_benchmark_returns = np.exp(np.log(benchmark_period_return + 1.0).diff()) - 1

  # Create tear sheet
  pf.create_full_tear_sheet(returns, positions=positions, transactions=transactions, benchmark_rets=daily_benchmark_returns)

start_date = pd.to_datetime('1993-1-1', utc=True)
end_date = pd.to_datetime('2018-1-1', utc=True)

results = run_algorithm(
    start = start_date, 
    end = end_date,
    initialize = initialize,
    analyze = analyze,
    handle_data = handle_data,
    capital_base = 10000,
    data_frequency = 'daily',
    bundle ='quandl')

ModuleNotFoundError: ignored

In [ ]:
ND

# 📈# **SAVE PERFORMANCE RESULTS TO CSV** **bold text**

In [ ]:
# Store performance results (equity curve) as csv file on Drive
perf.portfolio_value.to_csv('model_performance.csv')